In [8]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\saisa\Downloads\customer_transactions.csv"

# Data Cleaning
## Handling Missing Values
print("\nChecking for missing values:")
print(data.isnull().sum())

# Impute missing values in 'Quantity_Purchased' with the median
median_quantity = data['Quantity_Purchased'].median()
data['Quantity_Purchased'] = data['Quantity_Purchased'].fillna(median_quantity)

# Impute missing values in 'Transaction_Value' with the median
median_transaction_value = data['Transaction_Value'].median()
data['Transaction_Value'] = data['Transaction_Value'].fillna(median_transaction_value)

# Drop rows with more than 50% missing data
data = data.dropna(thresh=len(data.columns) * 0.5)

# Drop columns with more than 70% missing data
data = data.loc[:, data.isnull().mean() < 0.7]

print("\nData after handling missing values:")
print(data.info())

## Outlier Detection and Handling
def replace_outliers_with_median(df, column_name):
    """Replaces outliers in the specified column with the median value using the IQR method."""
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median_value = df[column_name].median()
    df[column_name] = np.where((df[column_name] < lower_bound) | (df[column_name] > upper_bound), median_value, df[column_name])

# Handle outliers for 'Transaction_Value'
replace_outliers_with_median(data, 'Transaction_Value')

# Handle extreme outliers and negative values for 'Delivery_Time'
data = data[data['Delivery_Time'] >= 0]  # Remove negative delivery times
replace_outliers_with_median(data, 'Delivery_Time')

print("\nData after handling outliers:")
print(data.describe())

## Data Transformation

### Categorical Variable Encoding
# Convert 'Payment_Method' to one-hot encoding
payment_dummies = pd.get_dummies(data['Payment_Method'], prefix='Payment')
data = pd.concat([data, payment_dummies], axis=1)

# Create a binary variable 'Return_Flag' for 'Is_Returned'
data['Return_Flag'] = data['Is_Returned'].apply(lambda x: 1 if x == 'Yes' else 0)
data['Is_Returned'] = data['Is_Returned'].astype('category')

print("\nData after categorical encoding:")
print(data.head())

### Feature Engineering
# Convert 'Transaction_Date' to datetime and create 'Transaction_Year'
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])
data['Transaction_Year'] = data['Transaction_Date'].dt.year

# Create 'High_Rating' feature
data['High_Rating'] = data['Customer_Rating'].apply(lambda x: 1 if x >= 4 else 0)

print("\nData after feature engineering:")
print(data.head())

## Save the cleaned data
data.to_csv('cleaned_customer_transactions.csv', index=False)
print("\nCleaned data saved to 'cleaned_customer_transactions.csv'")



Checking for missing values:
Transaction_ID             0
Customer_ID                0
Transaction_Date           0
Product_ID                 0
Quantity_Purchased         0
Transaction_Value          0
Payment_Method             0
Delivery_Time              0
Customer_Rating            0
Customer_Comments        249
Is_Returned                0
Payment_Bank Transfer      0
Payment_Cash               0
Payment_Credit Card        0
Payment_PayPal             0
Return_Flag                0
Transaction_Year           0
High_Rating                0
Payment_Bank Transfer      0
Payment_Cash               0
Payment_Credit Card        0
Payment_PayPal             0
Payment_Bank Transfer      0
Payment_Cash               0
Payment_Credit Card        0
Payment_PayPal             0
dtype: int64

Data after handling missing values:
<class 'pandas.core.frame.DataFrame'>
Index: 875 entries, 1 to 999
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  --